# AC297r Capstone Project
## Legendary team
### SEDMA testing
- This code mainly focus on multithreading test

# Async Server
### Start server from terminal, no need to execute this session

In [13]:
import subprocess
import os
import signal
p=subprocess.Popen(["./start_server.sh"], preexec_fn=os.setsid)

In [2]:
os.killpg(os.getpgid(p.pid), signal.SIGTERM)

ProcessLookupError: [Errno 3] No such process

# Multiple Client (multithread testing)
- Run through this test sequentially
- Look at terminal for logging that server creates

## 1. Preparation
- get file name of sample images
- import client class

In [12]:
from os import listdir
from os.path import isfile, join
mypath = "frontend/upload/you_images_all"
filenames = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [13]:
print(len(filenames))
print(filenames[1:5])

10277
['0096adf6bced11e19e4a12313813ffc0_7.jpg', '009d35d40be111e2949722000a1f90e1_7.jpg', '01ab7f5efea611e2ac7c22000a1f96a8_7.jpg', '01ecbe18b13111e38153128b961f8de0_8.jpg']


### Initialize client
- import the client class

In [3]:
import sys
sys.path.append('./server')
from baseClient import dbClient
from database import databaseAPI

## 2. Test image insertion from multiple clients
### Multithread insertion of images

In [4]:
images_per_group=5
def clientinsert(group):
    client = dbClient()
    for count in range(images_per_group):
        filenow = filenames[group*images_per_group+count+1]
        client.insertImage("you_images_all/"+filenow)
    return True

In [15]:
# single process
clientinsert(0)

True

In [5]:
# multi process
import multiprocessing
num=50
p = multiprocessing.Pool(num)
print(p.map(clientinsert,range(num)))


[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


## 3. Test insert model label from multiple clients
### Multithread insertModelLabel

In [8]:
# get all image_ids
clientST = dbClient()
image_ids=clientST.query("SELECT id FROM images limit 4000")
print(image_ids[:5])
print(len(image_ids))

[('0117dc99f07647d6',), ('01656de9d99333d2',), ('01ff831aff0432ec',), ('0376e50cf0e78c3b',), ('03ad165794770e97',)]
249


In [9]:
images_per_group=4
def clientinsertModelLabel(group):
    client = dbClient()
    for count in range(images_per_group):
        idnow = image_ids[group*images_per_group+count+1]
        GOT_VALUE = client.insertModelLabel(idnow[0], label=group)
    return GOT_VALUE

In [46]:
# single process
clientinsertModelLabel(4)

True

In [10]:
# multi process
import multiprocessing
num=50
p = multiprocessing.Pool(num)
print(p.map(clientinsertModelLabel,range(num)))


[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


In [10]:
# check inserted labels
image_id_label1=clientST.query("SELECT id, label FROM images")
print(len(image_id_label1))
print(image_id_label1[100:112])

249
[('73af8f723c844919', 0), ('534b57a87277411b', 0), ('5bb23311c33aec74', 0), ('57773c0335996589', 0), ('01ff831aff0432ec', 0), ('6f0da50685e170fb', 0), ('23d9dc23dd4e34a8', 0), ('7f9c7483634b8274', 0), ('99a857bca86747a1', 0), ('e3300e6f890eced3', 0), ('5154aeebb8b181d6', 0), ('0f87a4ca7b68c72a', 0)]


In [7]:
# check inserted labels
image_id_label=clientST.query("SELECT image_id, label FROM modelLabels")
print(len(image_id_label))
print(image_id_label[0:5])

0
[]


## 4. Test get RandomImageWithWeakLabel from multiple clients
### Multithread getRandomImageWithWeakLabel

In [4]:
from shutil import copyfile
import os
images_per_group=50
def clientgetRandomImageWithWeakLabel(group):
    # create subfolder
    dirnow = "frontend/upload/group"+str(group)
    if not os.path.exists(dirnow):
        os.makedirs(dirnow)
    # start to get random images
    client = dbClient()
    for count in range(images_per_group):
        imag_dic = client.getRandomImageWithWeakLabel()
        dst_now = dirnow + "/" + "g" + str(group) + "i" + str(count) + "_"+ imag_dic['path'].split('/')[-1]
        copyfile(imag_dic['path'], dst_now)
    return imag_dic

In [8]:
# single process
clientgetRandomImageWithWeakLabel(0)

True

In [5]:
# multi process
import multiprocessing
num=50
p = multiprocessing.Pool(num)
print(p.map(clientgetRandomImageWithWeakLabel,range(num)))

[{'id': 'a3714e887c13d3b9', 'labels': [43], 'path': '/Users/minglong/Desktop/CSE_2016/2nd_semester/AC297R/legendary/server/data/images/other/a3714e887c13d3b9.jpg'}, {'id': '6b69c49e98b8b499', 'labels': [26], 'path': '/Users/minglong/Desktop/CSE_2016/2nd_semester/AC297R/legendary/server/data/images/other/6b69c49e98b8b499.jpg'}, {'id': '5b9da3cc4502f26b', 'labels': [22], 'path': '/Users/minglong/Desktop/CSE_2016/2nd_semester/AC297R/legendary/server/data/images/other/5b9da3cc4502f26b.jpg'}, {'id': '8d49b3c9c739a638', 'labels': [37], 'path': '/Users/minglong/Desktop/CSE_2016/2nd_semester/AC297R/legendary/server/data/images/other/8d49b3c9c739a638.jpg'}, {'id': '01656de9d99333d2', 'labels': [0], 'path': '/Users/minglong/Desktop/CSE_2016/2nd_semester/AC297R/legendary/server/data/images/other/01656de9d99333d2.jpg'}, {'id': '4db17b87a4fc840e', 'labels': [18], 'path': '/Users/minglong/Desktop/CSE_2016/2nd_semester/AC297R/legendary/server/data/images/other/4db17b87a4fc840e.jpg'}, {'id': '8f83d25b

## 5. Test predict and insert from multiple clients
### Multithread predict_and_insert

In [25]:
images_per_group=1
def clientpredict_and_insert(group):
    client = dbClient()
    for count in range(images_per_group):
        filenow = filenames[group*images_per_group+count+1]
        # GOT_VALUE = client.predict_and_insert("./"+filenow)
        GOT_VALUE = client.predict_and_insert("you_images_all/" + filenow)
    return GOT_VALUE[0]

In [28]:
# single process
clientpredict_and_insert(0)

(0.93678659, 'amusement')

In [31]:
# multi process
import multiprocessing
num=50
p = multiprocessing.Pool(num)
kkk = p.map(clientpredict_and_insert,range(num))

In [34]:
kkk

[(0.93678659, 'amusement'),
 (0.39507085, 'amusement'),
 (0.93013132, 'amusement'),
 (0.67472547, 'contentment'),
 (0.24057609, 'amusement'),
 (0.94036657, 'amusement'),
 (0.51355696, 'amusement'),
 (0.57298118, 'amusement'),
 (0.50214636, 'contentment'),
 (0.63608772, 'amusement'),
 (0.4846873, 'contentment'),
 (0.78308022, 'contentment'),
 (0.81969178, 'amusement'),
 (0.81849784, 'amusement'),
 (0.72596604, 'amusement'),
 (0.75963759, 'contentment'),
 (0.90725046, 'amusement'),
 (0.88336599, 'amusement'),
 (0.73755699, 'amusement'),
 (0.25005716, 'sadness'),
 (0.36671564, 'excitement'),
 (0.5894205, 'awe'),
 (0.86518687, 'contentment'),
 (0.6370399, 'contentment'),
 (0.29111123, 'awe'),
 (0.62590128, 'amusement'),
 (0.40293154, 'contentment'),
 (0.93654972, 'amusement'),
 (0.96512526, 'amusement'),
 (0.80930763, 'amusement'),
 (0.93160796, 'amusement'),
 (0.27830276, 'amusement'),
 (0.25363567, 'contentment'),
 (0.90961301, 'amusement'),
 (0.94299346, 'amusement'),
 (0.93920183, 'amu

# The following is debugging code

### Print out db

In [8]:
#create db instance
from server.database import databaseAPI
db=databaseAPI('server/test.db','server/data')
print(db.printSchemas())

images:
  0: id(TEXT PRIMATY KEY) 
  1: path(TEXT) 
  2: label(INTEGER) 
  3: confidence(INTEGER) 
  4: source(TEXT) 
  5: comment(TEXT) 
labelType:
  0: id(INTEGER) *1 
  1: name(TEXT) 
modelLabels:
  0: id(INTEGER) *1 
  1: model(TEXT) 
  2: image_id(TEXT) 
  3: label(INTEGER) 
  4: confidence(REAL) 
models:
  0: name(TEXT) 
  1: path(TEXT) 
  2: accuracy(REAL) 



In [46]:
import multiprocessing
jobs=[]
p1 = multiprocessing.Process(target=clientinsert(1))
jobs.append(p1)
p1.start()


Process Process-44:
Traceback (most recent call last):
  File "/Applications/anaconda/envs/py35/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Applications/anaconda/envs/py35/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
TypeError: 'bool' object is not callable


In [8]:
from server.baseClient import dbClient
import sys
sys.path.append('./server')
from server.database import databaseAPI

In [11]:
import multiprocessing
def clientquery(x):
    client = dbClient()
    return(client.query("SELECT COUNT(*) FROM images"))

num=6
p = multiprocessing.Pool(num)
print(p.map(clientquery,range(num)))

['no such table: images', 'no such table: images', 'no such table: images', 'no such table: images', 'no such table: images', 'no such table: images']


In [13]:
def clientinsert(x):
    client = dbClient()
    return(client.insertImage(x))
jobs=[]
p1 = multiprocessing.Process(target=clientinsert("samples/img_from_tweet_7908_0.jpg"))
jobs.append(p1)
p1.start()


In [17]:
p1 = multiprocessing.Process(target=clientinsert("samples/img_from_tweet_9388_0.jpg"))
p1.start()

In [23]:
p1 = multiprocessing.Process(target=clientquery)
jobs.append(p1)
p1.start()
jobs

[<Process(Process-21, stopped)>,
 <Process(Process-29, stopped)>,
 <Process(Process-29, stopped)>,
 <Process(Process-30, stopped[1])>,
 <Process(Process-43, started)>]

Process Process-43:
Traceback (most recent call last):
  File "/Applications/anaconda/envs/py35/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Applications/anaconda/envs/py35/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
TypeError: clientquery() missing 1 required positional argument: 'x'


In [24]:
jobs

[<Process(Process-21, stopped)>,
 <Process(Process-29, stopped)>,
 <Process(Process-29, stopped)>,
 <Process(Process-30, stopped[1])>,
 <Process(Process-43, stopped[1])>]

# Server Client

In [10]:
#create client
from server.baseClient import dbClient
client = dbClient()

In [18]:
#insert image, do prediction and insert label
print(client.predict_and_insert('test.jpeg'))

INFO:root:client: sending predict_and_insert request


True


In [85]:
kkk = client.getRandomImageWithWeakLabel()
type(kkk)
kkk

{'id': '834172f9d8bc325e',
 'labels': [1, 3],
 'path': '/Users/minglong/Desktop/CSE_2016/2nd_semester/AC297R/legendary/server/data/images/other/834172f9d8bc325e.jpg'}

In [81]:
kkk['labels']

[0, 2]

In [75]:
kkk['path'].split('/')[-1]

'4b4f4313f13c8dcc.jpg'

In [15]:
client.predict_and_insert("wayward_pines_images/"+filenames[5])

"'NoneType' object has no attribute 'split'"

In [ ]:
        filenow = filenames[group*images_per_group+count+1]
        client.insertImage("wayward_pines_images/"+filenow)

In [7]:
filenames[3]

'img_from_tweet_56168_0.jpg'

In [56]:
image_id_label=client.query("SELECT image_id, label FROM modelLabels")

In [60]:
image_id_label[2000:2100]

[('6b4bc46468bcb696', 3),
 ('6b5318dcc665263d', 3),
 ('6b78a664c66a17f0', 3),
 ('6b8ba960c4a91e7e', 3),
 ('6bccd737868cd209', 3),
 ('6bd318dcc6672634', 3),
 ('6d55e589d2bcd0a1', 3),
 ('6d58a0461fb7623d', 3),
 ('6d939312a56dd94a', 3),
 ('6ddbb22cc494565a', 3),
 ('6ddfef20900c1477', 3),
 ('6de9dc865683433c', 3),
 ('6df8d20d96963233', 3),
 ('6f17862e1e9a1c4b', 3),
 ('6f574ca3944cb1b4', 3),
 ('6f8694c93659d235', 3),
 ('6f9191ba46c6906f', 3),
 ('6f91c817dde428a9', 3),
 ('6fe31e903817dc58', 3),
 ('714496356d9abc39', 3),
 ('71459475699abc39', 3),
 ('714b9e7061c72799', 3),
 ('716755a37393ba10', 3),
 ('717545aac97091ee', 3),
 ('717b8e81a316d987', 3),
 ('718a75635ef5a284', 3),
 ('71ce8dd689462b99', 3),
 ('71d78b2b74d48329', 3),
 ('71e346f9460d3b1c', 3),
 ('71e9a8ad873594a5', 3),
 ('71f40f0f4f207ac3', 3),
 ('71f70f0b21b1b18e', 3),
 ('731b0ccc3367cc8b', 3),
 ('73436773714c33a2', 3),
 ('7347236631cc3ab6', 3),
 ('73638ccc99cc9ccc', 3),
 ('73639c4c99ce9c8c', 3),
 ('736d96cccc99cc11', 3),
 ('7389b9b10

In [22]:
#query
print(client.query("SELECT id FROM images"))

[(1331,)]


In [19]:
print(client.query("SELECT * FROM modelLabels"))

INFO:root:client: sending query


[(1, 'testing', 'a9535674ec565c0e', 8, 0.0), (2, 'testing', 'a9535674ec565c0e', 3, 0.0)]


In [6]:
#get random. only work when there are weak labels
print(client.getRandomImageWithWeakLabel())

INFO:root:client: requesting weak label


{'id': '23646c9bd4e99c9a', 'path': '/Users/wangmeili/Documents/sedma/server/data/images/other/23646c9bd4e99c9a.jpg', 'labels': [3, 4]}


In [16]:
#insert image but no prediction
print(client.insertImage('test.jpeg'))

a9535674ec565c0e duplicated entry, insert reverted


# Data Base Client

In [7]:
#create db instance
from server.database import databaseAPI
db=databaseAPI('server/test.db','server/data')

In [8]:
print(db.query_meta("SELECT COUNT(*) FROM images"))

[(1450,)]


In [4]:
print(db.getRandomImageWithWeakLabel())

{'labels': [1, 6], 'id': 'bb4ce465d2962c8e', 'path': '/Users/wangmeili/Documents/sedma/server/data/images/other/bb4ce465d2962c8e.jpg'}


In [5]:
print(db.query_meta("SELECT id FROM images WHERE label=0"))

[('a9535674ec565c0e',), ('a15ee83ccb9231a7',), ('57ccab23b1a6cc4a',), ('61031bb2f5d90ec7',), ('5f81e81ac6e73139',), ('d319632c78c38e9b',), ('87968ec64c636a73',), ('b5a3164c2d1f93e1',), ('23646c9bd4e99c9a',), ('ef619b1616ba61c1',), ('f3610c9b63cc6699',), ('8f0738cf44f946e4',), ('87b6b3c339266319',), ('91b76e482678b867',), ('1fc9e53319c66d06',), ('67399c6d07499c36',), ('a308d6f2a763989d',), ('6b90386f90abc6f8',), ('bd3bc34db941e051',), ('af765cf05ca184a9',), ('2939dac7a46825ed',), ('774cd3f9102941af',), ('5749ada1a139b996',), ('358306fccdc1cbc6',), ('75f206439b182fe3',), ('ad6f411badd2881e',), ('5f93a9893939b238',), ('d78d60396f511b92',), ('bb4ce465d2962c8e',), ('99b20c3ef46c96cc',), ('99666ed8625ea42d',), ('3b61609ece08997f',), ('83d353786d8e868b',), ('d783c63423e7722c',), ('17e7e405143c7b5a',), ('e3e3850505727e6e',), ('1fcbd1511bab2625',), ('5b31b20eec093eec',), ('017d470df094f8f3',), ('8f654c9d771f8043',), ('d720fc0f07f1391c',), ('1db1a7854846dbe9',), ('770a90aa1d751bc7',), ('5b6e2713

In [6]:
db.insertMultipleImagesParallel('tempimages')

In [14]:
db.removeModelLabel(image_id='a9535674ec565c0e duplicated entry, insert reverted')

True

In [6]:
print(db.query_meta("SELECT path, id from images"))

[('data/images/other/00767f1797130301.jpeg', '00767f1797130301'), ('data/images/other/787e7c7878707070.jpeg', '787e7c7878707070'), ('data/images/other/3f3e3e3e3e3e3e20.jpeg', '3f3e3e3e3e3e3e20'), ('data/images/other/027e7e7e7e7e7000.jpeg', '027e7e7e7e7e7000'), ('data/images/other/007f7f7e60400000.jpeg', '007f7f7e60400000'), ('data/images/other/01ffffff1f1f0f00.jpeg', '01ffffff1f1f0f00')]


In [4]:
db.synchronize()

2017-03-16 21:33:24,946 - root - INFO - checking images
2017-03-16 21:33:24,949 - root - WARNING - 3f3e3e3e3e3e3e20 not in file system
2017-03-16 21:33:24,953 - root - WARNING - 00767f1797130301 not in file system
2017-03-16 21:33:24,956 - root - WARNING - 027e7e7e7e7e7000 not in file system
2017-03-16 21:33:24,958 - root - WARNING - 01ffffff1f1f0f00 not in file system
2017-03-16 21:33:24,960 - root - WARNING - 007f7f7e60400000 not in file system
2017-03-16 21:33:24,963 - root - WARNING - 787e7c7878707070 not in file system
2017-03-16 21:33:24,966 - root - INFO - checking models
2017-03-16 21:33:24,968 - root - INFO - Done


In [12]:
print(db.query_meta('SELECT * FROM images where id = "8080707efec1c3f7"')==[('8080707efec1c3f7', 'data/images/other/8080707efec1c3f7.jpg', 'NULL', 5, 'other', 'NULL')])

True


In [21]:
db.removeImage('007f7f7e60400000')

In [5]:
print(db.printSchemas())

images:
  0: id(TEXT PRIMATY KEY) 
  1: path(TEXT) 
  2: label(INTEGER) 
  3: confidence(INTEGER) 
  4: source(TEXT) 
  5: comment(TEXT) 
labelType:
  0: id(INTEGER) *1 
  1: name(TEXT) 
modelLabels:
  0: id(INTEGER) *1 
  1: model(TEXT) 
  2: image_id(TEXT) 
  3: label(INTEGER) 
  4: confidence(REAL) 
models:
  0: name(TEXT) 
  1: path(TEXT) 
  2: accuracy(REAL) 



In [14]:
db.printSchemas()

images:
  0: id(TEXT PRIMATY KEY)
  1: path(TEXT)
  2: label(TEXT)
  3: confidence(INTEGER)
  4: source(TEXT)
  5: comment(TEXT)
modelLabels:
  0: id(INTEGER PRIMATY KEY)
  1: model(TEXT)
  2: image_id(TEXT)
  3: label(TEXT)
  4: confidence(REAL)
models:
  0: name(TEXT)
  1: path(TEXT)
  2: accuracy(REAL)


# Image

In [29]:
#create new image
from PIL import Image
im=Image.new("RGB", (512, 512))
data=[(i//3,0,i//3) for i in range(512)]
im.putdata(data)

print(database.utility.hashImage(im))
#im.show()

c000000000000000


In [8]:
# hash image
from PIL import Image
import imagehash
hashed = imagehash.phash(Image.open('test.jpeg'))
print(hashed)

a9535674ec565c0e
